In [1]:
import os

from qiskit.transpiler.passes.optimization import *

from pytket import OpType
from pytket.passes import *

from FT_optimization_generate_data import *

SINGLE_QUBIT_AND_CX_QISKIT_STDGATES = [
    "x",
    "y",
    "z",
    "h",
    "s",
    "sdg",
    "t",
    "tdg",
    "sx",
    "rx",
    "ry",
    "rz",
    "cx",
]

SINGLE_QUBIT_AND_CX_TKET_STDGATES = {
    OpType.Rx,
    OpType.Ry,
    OpType.Rz,
    OpType.X,
    OpType.Y,
    OpType.Z,
    OpType.H,
    OpType.S,
    OpType.Sdg,
    OpType.T,
    OpType.Tdg,
    OpType.SX,
    OpType.CX
}

# Qiskit

In [2]:
circuit_folder = "MQTBenchQiskit"

benchmarks = [f.replace(".qasm", "") for f in os.listdir(circuit_folder) if f.endswith(".qasm")]
benchmark_names = [name.split("_indep_qiskit_")[0] for name in benchmarks]
benchmarks.sort()

basis_gates = SINGLE_QUBIT_AND_CX_QISKIT_STDGATES

transpiler_passes = [
    [Optimize1qGatesDecomposition(basis=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [Optimize1qGatesSimpleCommutation(basis=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [CommutationAnalysis(), CommutativeCancellation(basis_gates=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [HoareOptimizer(), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [TemplateOptimization(), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
]

In [3]:
generate_data(
    f"results_qiskit.xlsx",
    benchmarks, transpiler_passes,
    None,
    'qiskit',
)

  0%|          | 0/12 [00:00<?, ?it/s]

/Users/tobiasforster/Documents/Projects/FT Optimization Public Code/mqt-problemsolver/resource_estimation/FT_optimization_generate_data.py:172: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_existing = pd.concat([df_existing, new_data], ignore_index=True)
 92%|█████████▏| 11/12 [13:49<01:56, 116.69s/it]

Error processing qwalk-v-chain_indep_qiskit_7 [<qiskit.transpiler.passes.optimization.hoare_opt.HoareOptimizer object at 0x15e48c410>, <qiskit.transpiler.passes.basis.basis_translator.BasisTranslator object at 0x15e30ede0>]
'Node not in DAG'


100%|██████████| 12/12 [13:53<00:00, 69.45s/it] 

Results saved and updated in results_qiskit.xlsx


# Tket

In [ ]:
circuit_folder = "MQTBenchTKET"

benchmarks = [f.replace(".qasm", "") for f in os.listdir(circuit_folder) if f.endswith(".qasm")]
benchmark_names = [name.split("_indep_tket_")[0] for name in benchmarks]
benchmarks.sort()

transpiler_passes = [
    RemoveRedundancies(),
    CliffordSimp(),
    FullPeepholeOptimise(),
    EulerAngleReduction(OpType.Rx, OpType.Ry),
    KAKDecomposition(),
    RepeatPass(SequencePass([CommuteThroughMultis(), RemoveRedundancies()]))
]

transpiler_pass_names = [
    "RemoveRedundancies",
    "CliffordSimp",
    "FullPeepholeOptimise",
    "EulerAngleReduction",
    "KAKDecomposition",
    "RepeatPass"
]

In [ ]:
generate_data(
    f"results_tket.xlsx",
    benchmarks,
    transpiler_passes,
    transpiler_pass_names,
    'tket'
)